# Test data

Uso el dataset de Daimler. En mi PC descomprimí las 5 partes en la misma carpeta.
En caso de querer utlizar solo la primer parte, setear SUBSET en un número menor 4000.
También utilizo el Ground Truth provisto por Daimler para checkear que la predicción sea correcta.


To do list:
* Entrenar el clasificador cuando es un falso positivo.
* Guardar los hogs en el HDF5

In [ ]:
import skimage.io
from sklearn import svm
from sklearn.externals import joblib
from skimage.feature import hog
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches


CHECKPOINT_PATH = '/home/beto0607/Facu/Pedestrians/Datasets/svmCheckpoint.pkl'  # Path donde se guarda el SVM ya entrenado
HDF5_PATH = '/home/beto0607/Facu/Pedestrians/Datasets/datasets.h5'

#Si es 0, toma las 21.790 imagenes
SUBSET = 250
DAIMLER_PATHS = {
    #Tengo las 5 partes descomprimidas en esta carpeta, son 21.790
    'Test_Data': '/home/beto0607/Facu/Pedestrians/Datasets/Daimler/DaimlerBenchmark/Data/TestData/',
    'Ground_Truth':'/home/beto0607/Facu/Pedestrians/Datasets/Daimler/GroundTruth/DaimlerBenchmark/GroundTruth/GroundTruth2D.db'
}
DAIMLER_OBJECTS_REFERENCE = {
    0: 'Pedestrian',
    1: 'Bicyclist',
    2: 'Motocyclist',
    10: 'Pedestrian group',
    255: 'Partially hidden'
}
FINAL_SIZE = [96, 48] #para Resize

IMAGE_PATH = './Test_image.pgm'

WINDOW_SIZE = ((96, 48), (144, 72)) #Diferentes tamaños de ventana
WINDOW_STEP_X = 24 #el paso entre ventana y ventana, eje X
WINDOW_STEP_Y = 48 #el paso entre ventana y ventana, eje Y


def JoinPaths(base, path):
    return os.path.join(base,path)

def SaveClassifier(classifier):
    joblib.dump(classifier, CHECKPOINT_PATH)
    
def LoadCheckpoint():
    return joblib.load(CHECKPOINT_PATH)

def LoadImage(image_path):
    return skimage.io.imread(image_path, as_gray = True)
    
def Predict(classifier, img):#Predice imagen por imagen o una lista de imagenes
    if (type(img) is list) or (type(img) is tuple):
        return classifier.predict(hog(img, block_norm='L2-Hys', transform_sqrt=True))
    else:
        return classifier.predict([hog(img, block_norm='L2-Hys', transform_sqrt=True)])[0]

def DaimlerGroundTruth():
    
    pedestrians = []
    
    cLines = 0
    ground = open(DAIMLER_PATHS['Ground_Truth'], 'r')
    #A ignorar - HEADER
    ground.readline()
    ground.readline()
    ground.readline()
    ground.readline()
    for line in ground:
        if(line[0] == ';'):
            imageFile = ground.readline().split('\n')[0]
            #A ignorar - Shape
            ground.readline()
            #A ignorar - Object2Dor3D
            (object2Dor3D, objectCant) = map(int, ground.readline().split(' '))
            #objectCant = int(objectCant)
            boundingBoxes = []
            for i in range(0, objectCant):
                #Tipo de objeto encontrado(basado en el dict de arriba)
                objectType = int(ground.readline().split(' ')[1])

                #A ignorar - Track ID & Unique ID
                a = ground.readline()
                #A ignorar - Object confidence
                b = ground.readline()

                #Shape del BoundingBox
                (x_min, y_min, x_max, y_max) = map( int, ground.readline().split(' '))

                #A ignorar - Boolean - Has bounding box available
                ground.readline()

                boundingBoxes.append({
                    'type': objectType,
                    'x_min': x_min,
                    'x_max': x_max,
                    'y_min': y_min,
                    'y_max': y_max,
                    'width': x_max - x_min,
                    'height': y_max - y_min
                })

            pedestrians.append({
                'imageFile': imageFile,
                'quantity': objectCant,
                'boundingBoxes': boundingBoxes
            })
        cLines+=1
        if(cLines==SUBSET):
            break
    ground.close()

    return pedestrians, len(pedestrians)

def CropAndResize(image, x,y, width, height):
    img_crop = image[y:y+height, x:x+width]
    img_crop = skimage.transform.resize(img_crop, FINAL_SIZE)
    return img_crop

def IsRectInOtherRect(rect,otherRect):
    #Rect[0] = x;Rect[1] = y;Rect[2] = width;Rect[3] = height;
    return (otherRect[0] <= rect[0] and 
            otherRect[1] <= rect[1] and 
            otherRect[0]+otherRect[2] >= rect[0]+rect[2] and
            otherRect[1]+otherRect[3] >= rect[1]+rect[3])

def CheckRectInsideBoundingBox(rect, p):
    #print(p)
    if(p['quantity'] > 0):
        for  i in range(0, p['quantity'] ) :
            b = p['boundingBoxes'][i]
            if(IsRectInOtherRect(rect, (b['x_min'],b['y_min'],b['width'],b['height']) )):
                return True
            if(IsRectInOtherRect((b['x_min'],b['y_min'],b['width'],b['height']), rect )):
                return True
            
    return False

def PredictionsForImageAndWindowSize(classifier, image, wSize, p):
    correctas = 0
    incorrectas = 0
    boundingBoxes = []
    x = 0
    y = 0
    while(y >=0 and (y + wSize[1] <= image.shape[0])):
        while(x >=0 and (x + wSize[0] <= image.shape[1])):
            prediction = Predict(classifier, CropAndResize(image, x,y, wSize[0], wSize[1]))
            if(prediction == 1):
                if(CheckRectInsideBoundingBox((x,y,wSize[0], wSize[1]), p )):
                    correctas+=1
                    #print('Prediccion correcta')
                    boundingBoxes.append({
                        'x': x,
                        'y': y,
                        'width': wSize[0],
                        'height': wSize[1],
                        'patch': patches.Rectangle((x,y),wSize[0], wSize[1],linewidth=2,edgecolor='g',facecolor='none')
                    })
                else:
                    incorrectas+=1
                    #print('Prediccion incorrecta')
                
                
            x += WINDOW_STEP_X
        y += WINDOW_STEP_Y
        x = 0
    return boundingBoxes, correctas, incorrectas
    
def main():
    #Cargo el GroundTruth de Daimler
    pedestrians, size = DaimlerGroundTruth()
    
    #Cargo el Checkpoint del Clasificador
    classifier = LoadCheckpoint()
        
    statistics = {
        'correctas': 0,
        'incorrectas': 0,
        'cantidad': 0
    }
        
    for p in pedestrians:
        
        #Prueba solamente las imaágenes que si tienen peatones
        if(p['quantity'] == 0):

            continue
        
        statistics['cantidad']+=1
        image = LoadImage(JoinPaths(DAIMLER_PATHS['Test_Data'], p['imageFile']))

        
        #fig,ax = plt.subplots(1)
        #ax.imshow(image, cmap='gray')
        
        for index in range(0,  len(WINDOW_SIZE)) :    
            
            predictions, corr, incorr = PredictionsForImageAndWindowSize(classifier, image, WINDOW_SIZE[index], p)
            statistics['correctas']+=corr
            statistics['incorrectas']+=incorr
        
#        break
        
    print(statistics)
    SaveClassifier(classifier)

main()